In [16]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [17]:
from crewai import Agent, Task, Crew


In [18]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()


In [19]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [20]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [21]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [22]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [23]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [24]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)


In [25]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [26]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [27]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


In [28]:
from crewai import Crew, Process # Process class let's you define different ways for agents to work together
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)


2024-10-31 15:11:18,447 - 8382724160 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [29]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'ERJ',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}


In [30]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)


 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (ERJ). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for ERJ to identify trends and predict market movements. I should delegate tasks and ask questions to the team members who can provide valuable insights.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for ERJ using statistical modeling and machine learning techniques", "context": "The goal is to identify trends and predict market movements for ERJ. Make sure to provide insights and alerts about significant market opportunities or threats."}

> Entering new CrewAgentExecutor chain...
Thought: To monitor and analyze market data for ERJ (Embraer S.A.),

In [31]:
from IPython.display import Markdown
Markdown(result)


Risk Analysis Report for ERJ Trading Strategies and Execution Plans:

Introduction:
Embraer (ERJ) is a significant player in the aviation industry, positioned as a viable stock for growth-focused investors. The company operates in multiple segments, including commercial, executive, and defense sectors. ERJ is rated as a #3 (Hold) on the Zacks Rank with a VGM Score of B, indicating a balanced opportunity for value, growth, and momentum investors. This report aims to provide a comprehensive analysis of potential risks associated with ERJ's proposed trading strategies and execution plans, along with mitigation recommendations.

Risk Analysis:
1. Revenue and Order Backlog:
ERJ's 15% revenue increase and high order backlog signal strong demand for its products. However, high order backlogs pose operational risks due to potential supply chain issues or increased production costs. Mitigation: Regular monitoring of supply chain performance and production costs to address any operational challenges.

2. Supply Chain and Currency Volatility:
Challenges related to supply chain disruptions and currency fluctuations introduce financial and operational risks for ERJ. Mitigation: Implement risk management strategies to mitigate the impact of supply chain disruptions and currency volatility on delivery timelines and cost targets.

3. Geopolitical and Sector-Specific Developments:
ERJ's international market presence exposes it to geopolitical risks. Changes in political stability, regulatory environments, or trade policies could impact operations and profitability. Mitigation: Stay informed about geopolitical developments and diversify market presence to minimize risks.

4. Market Performance and Competitive Landscape:
ERJ's stock performance shows volatility, reflecting broader market trends and investor sentiments. Competition from other aerospace manufacturers poses a risk to market share and pricing power. Mitigation: Monitor market trends and competitive landscape to make informed trading decisions.

5. Innovative Ventures and Partnerships:
ERJ's strategic collaborations in satellite communications and 'flying cars' indicate a focus on innovation. While these ventures offer growth opportunities, they carry risks associated with unproven markets and technologies. Mitigation: Conduct thorough market research and risk assessments before entering new ventures.

Conclusion:
ERJ's trading strategy should include careful monitoring of revenue, order backlog, supply chain performance, currency fluctuations, geopolitical developments, market trends, competition, and innovative ventures. Mitigation strategies should be implemented to address potential risks and align trading decisions with ERJ's risk tolerance and market conditions. By proactively managing risks, investors can optimize returns and position ERJ for sustainable growth in the aviation industry.